In [17]:
!pip install keras-rl

/bin/bash: /home/padmapriya/miniconda3/envs/RLforQEC/lib/libtinfo.so.6: no version information available (required by /bin/bash)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for keras-rl: filename=keras_rl-0.4.2-py3-none-any.whl size=48363 sha256=c3ae4e7598b96838b6bbcb00dfcb3def15c1a41b57d41bea020f686029ddd03e
  Stored in directory: /home/padmapriya/.cache/pip/wheels/44/fb/c7/96ef7b9a9ded58c3300148907fd398167f44965686e7d04e0f
Successfully built keras-rl


In [2]:

from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
import random

In [1]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [18]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [7]:
class QECEnvOne(Env):
    def __init__(self):
        # the action spacee consists of 32 discrete actions - apply pulse or don't apply pulse at each one of 5 steps
        self.action_space = Discrete(32)
        # the observation space consists of an array of values betwene 0 and 1 that indicate fidelity
        self.observation_space = Box(low = np.array([0]), high = np.array([1]))
        
        
        self.state = random.randint(0,1)
        
        
        # the number of steps is 5
        self.seq_len = 5
        
    def step(self, action):
        # for now, the state is randomly updated based on the action taken
        self.state = random.randint(0,1)
        self.seq_len -= 1
        
        if self.state >=0.5:
            reward = 1
        else: 
            reward = -1
            
        if self.seq_len <=0:
            done = True
        else:
            done = False
        info = {}
            
        return self.state, reward, done, info
            
            
    def reset(self):
        self.state = random.randint(0,1)
        self.seq_len = 5
        return self.state
    

In [8]:
env = QECEnvOne()

In [9]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))

Episode:1 Score:-3
Episode:2 Score:-3
Episode:3 Score:1
Episode:4 Score:1
Episode:5 Score:1
Episode:6 Score:-1
Episode:7 Score:3
Episode:8 Score:-3
Episode:9 Score:1
Episode:10 Score:-1


array([0.9877322], dtype=float32)

In [11]:
states = env.observation_space.shape
actions = env.action_space.n

In [13]:
def build_model(states, actions):
    model = Sequential()    
    model.add(Dense(24, activation='relu', input_shape=states))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [15]:
model = build_model(states, actions)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 24)                48        
                                                                 
 dense_1 (Dense)             (None, 24)                600       
                                                                 
 dense_2 (Dense)             (None, 32)                800       
                                                                 
Total params: 1,448
Trainable params: 1,448
Non-trainable params: 0
_________________________________________________________________


In [19]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [20]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

TypeError: Keras symbolic inputs/outputs do not implement `__len__`. You may be trying to pass Keras symbolic inputs/outputs to a TF API that does not register dispatching, preventing Keras from automatically converting the API call to a lambda layer in the Functional Model. This error will also get raised if you try asserting a symbolic input/output directly.

In [21]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

NameError: name 'dqn' is not defined